# Emprical Cumulative Distribution Function on the GPU

<a href="http://www.aleagpu.com"><img style="float: right;" height="200" width="200" src="http://www.aleagpu.com/images/cube.svg"></a>

In this notebook we show how to use the GPU seamlessly in a scripting mode with <a href="http://www.aleagpu.com">Alea GPU V3</a> to calculate the empirical cumulative distribution function for a very large sample size. The prurpose of this notebook is to demonstrate the new highly efficient GPU primitives that will come with our next release 3.1 of Alea GPU, such as highly optimized parallel sort, scan, reduce, copy if, stream compactification, merge, join, etc. 

The empirical cumulative distribution function $\hat{F}_n(t)$ for the samples $x_1, \ldots, x_n$ is defined as

$${\hat  F}_{n}(t)={\frac  {{\mbox{number of elements in the sample}}\leq t}n}={\frac  {1}{n}}\sum _{{i=1}}^{n}{\mathbf  {1}}_{{x_{i}\leq t}}$$

It is an estimator of the true distribution function from which the samples $x_1, \ldots, x_n$ are generated. More details can be found on [Wikipedia](https://en.wikipedia.org/wiki/Empirical_distribution_function).



## Let's Get Started!

We first load the newest version 3.1 beta NuGet packages of <a href="http://www.aleagpu.com">Alea GPU</a>. You can run Alea GPU free on any CUDA capable GeForce or Quadro GPU. In case you want to run in on an enterprise GPU such as a Tesla K80 or P100 you need a <a href="http://www.quantalea.com/license">license</a>. 


In [1]:
#load "Paket.fsx"
Paket.Version [ ("Alea", "3.1.0-beta1") ]
Paket.Package [ "NUnit" ]

<null>

In [2]:
#load "packages/Alea/Alea.fsx"
#r "packages/Alea/lib/net45/Alea.Parallel.dll" 
#r "packages/NUnit/lib/net45/nunit.framework.dll"

In [ ]:
#load "XPlot.Plotly.Paket.fsx"
#load "XPlot.Plotly.fsx"
open XPlot.Plotly

## Inspect GPU 

We first check what GPU is available. As we only need one GPU for our computations, we select the default GPU.

In [ ]:
open System
open Alea
open Alea.CSharp
open Alea.Parallel

let gpu = Gpu.Default
sprintf "Gpu is %A, GPU DRAM is %.3f GB, Process is %d bit"
    gpu
    ((float) gpu.Device.TotalMemory / 1024. / 1024. / 1024.)
    (IntPtr.Size * 8)

## Binary Search

To sample the empirical cumulative distribution function we use binary search. We code a binary search function that will be used on the CPU and the GPU, giving us full code reuse. In order to be able to compile it to run on the GPU, we add the `[ReflectedDefinition]` attribute, so that the F# compiler generates the quotation for this function.

In [ ]:
[<ReflectedDefinition>]
let inline binarySearch (n:int) (v:int -> 'T) (x:'T) =
    let mutable l = 0
    let mutable u = n - 1
    while u - 1 > l do
        let m = int((uint32(l) + uint32(u)) >>> 1)
        if x < (v m) then u <- m
        else l <- m
    l

## CPU Version

We create a function to sample data and sample the empirical cumulative distribution function on a grid of points for plotting. Here is the CPU version:

In [ ]:
let ecdfCpu numSamples numPlotPoints (gen:float[] -> unit) =
    let numbers = Array.zeroCreate<float> numSamples
    let plotX = Array.zeroCreate<float> numPlotPoints
    let plotY = Array.zeroCreate<float> numPlotPoints
    
    gen numbers
    
    // start measuring time
    GC.Collect()
    GC.WaitForPendingFinalizers()
    let t = System.Diagnostics.Stopwatch.StartNew()
    
    // first sort it
    Array.sortInPlace numbers
    
    // grab min max value
    let min = numbers.[0]
    let max = numbers.[numSamples - 1]
    let dist = max - min

    // binary search to create ecdf
    for i = 0 to numPlotPoints - 1 do
        let x = min + (dist / (float numPlotPoints)) * (float i)
        let v i = numbers.[i]
        let y = binarySearch numSamples v x
        let y = (float y) / (float numSamples)
        plotX.[i] <- x
        plotY.[i] <- y
        
    t.Stop()
    
    // to get more accurate timings, we keep these array alive so we are not interrupted by GC collection
    GC.KeepAlive(numbers)
    GC.KeepAlive(plotX)    
    GC.KeepAlive(plotY)
    
    plotX, plotY, t.Elapsed 

## GPU Version

The GPU implementation relies on the new highly efficient GPU sort primitive, which we provide with Alea GPU 3.1 beta and a new session concept, which simplifies the API by hiding all the requires scratch pad memory allocation and management:

In [ ]:
let ecdfGpu numSamples numPlotPoints (gen:Session -> float[] -> unit) =
    // create a GPU session, which manages the temporary memory required by the different algrotihms.
    use session = new Session(gpu)
    
    let numbers = session.Allocate<float>(numSamples)
    let minmax = session.Allocate<float>(2)
    let plotX = session.Allocate<float>(numPlotPoints)
    let plotY = session.Allocate<float>(numPlotPoints)

    gen session numbers
    
    // start measuring time, we synchonize gpu first
    session.Gpu.Synchronize()
    GC.Collect()
    GC.WaitForPendingFinalizers()
    let t = System.Diagnostics.Stopwatch.StartNew()
    
    // first sort it
    session.Sort((fun a b -> a < b), numbers, numbers)
    
    // grab min max value
    session.Gpu.Launch((fun () ->
            minmax.[0] <- numbers.[0]
            minmax.[1] <- numbers.[numSamples - 1]),
        LaunchParam(1, 1))
        
    session.For(0, numPlotPoints, (fun i ->
        let min = minmax.[0]
        let max = minmax.[1]
        let dist = max - min
        let x = min + (dist / (float numPlotPoints)) * (float i)
        let v i = numbers.[i]
        let y = binarySearch numSamples v x
        let y = (float y) / (float numSamples)
        plotX.[i] <- x
        plotY.[i] <- y
    ))
        
    // synchornize gpu then stop timer
    gpu.Synchronize()
    t.Stop()
    
    // to get more accurate timings, we keep these array alive so we are not interrupted by GC collection
    GC.KeepAlive(numbers)
    GC.KeepAlive(minmax)
    GC.KeepAlive(plotX)
    GC.KeepAlive(plotY)

    Gpu.CopyToHost(plotX), Gpu.CopyToHost(plotY), t.Elapsed

## Run and Visualize 

We create sample generators for the normal and log-normal distribution using parallel random number generators from cuRand. Note that these generators also can be used to run on the CPU.

In [ ]:
let genGpuUniform (session:Session) (numbers:float[]) =
    session.RandomUniform(numbers, 0UL)
    
let genGpuNormal (session:Session) (numbers:float[]) =
    session.RandomNormal(numbers, 0.0, 1.0, 0UL)
    
let genGpuLogNormal (session:Session) (numbers:float[]) =
    session.RandomLogNormal(numbers, 0.0, 1.0, 0UL)
    
let genCpuUniform (data:float[]) =
    use rng = cuRAND.Generator.CreateCpu(cuRAND.RngType.PSEUDO_DEFAULT)
    rng.SetPseudoRandomGeneratorSeed(42UL)
    rng.SetGeneratorOffset(0UL)
    rng.GenerateUniform(data)
    
let genCpuNormal (data:float[]) =
    use rng = cuRAND.Generator.CreateCpu(cuRAND.RngType.PSEUDO_DEFAULT)
    rng.SetPseudoRandomGeneratorSeed(42UL)
    rng.SetGeneratorOffset(0UL)
    rng.GenerateNormal(data, 0.0, 1.0)
    
let genCpuLogNormal (data:float[]) =
    use rng = cuRAND.Generator.CreateCpu(cuRAND.RngType.PSEUDO_DEFAULT)
    rng.SetPseudoRandomGeneratorSeed(42UL)
    rng.SetGeneratorOffset(0UL)
    rng.GenerateLogNormal(data, 0.0, 1.0)  
    
let layout title x y= 
    Layout(title = title, 
           xaxis=Xaxis(title = x, showgrid = false, zeroline = false), 
           yaxis=Yaxis(title = y, showline = false), 
           showlegend = true)    

### Empirical Cummulative Distribution Function for the Normal Distribution

Let's estimate the empirical cumulative distribution function on the CPU and GPU for the normal distribution. Because we are only interested in the visualization, we choose a moderate sample size. 1MB gives us already sufficient accuracy.

In [ ]:
let numSamples = 1024*1024  
let numPlotPoints = 1000

In [ ]:
let x, y, _ = ecdfCpu numSamples numPlotPoints genCpuNormal
Scatter(name = "Normal", x = x, y = y, mode = "lines")
|> Chart.Plot 
|> Chart.WithLayout (layout "Empirical Cumulative Distribution Function on CPU" "x" "probability")

In [42]:
let x, y, _ = ecdfGpu numSamples numPlotPoints genGpuNormal
Scatter(name = "Normal", x = x, y = y, mode = "lines") 
|> Chart.Plot 
|> Chart.WithLayout (layout "Empirical Cumulative Distribution Function on GPU" "x" "probability")

### Empirical Cummulative Distribution Function for the Log-Normal Distribution

We repeate the experiment for the log-normal distribution. Note that it has support on $[0, \infty)$ only.

In [43]:
let x, y, _ = ecdfCpu numSamples numPlotPoints genCpuLogNormal
Scatter(name = "Log Normal", x = x, y = y, mode = "lines")
|> Chart.Plot 
|> Chart.WithLayout (layout "Empirical Cumulative Distribution Function on CPU" "x" "probability")

In [44]:
let x, y, _ = ecdfGpu numSamples numPlotPoints genGpuLogNormal
Scatter(name = "Log Normal", x = x, y = y, mode = "lines")
|> Chart.Plot 
|> Chart.WithLayout (layout "Empirical Cumulative Distribution Function on GPU" "x" "probability")

## Timings

The interesting question is of course how much faster the GPU does the job. Let's perform some timings on a larger sample set. Adjust the size according to the device memory of your GPU. 

In [ ]:
let numSamples = 1024*1024*10 // for small GPU please choose smaller count, say 10MB or 100MB
let _, _, cpuTime = ecdfCpu numSamples 1000 genCpuNormal
let _, _, gpuTime = ecdfGpu numSamples 1000 genGpuNormal

type data = { Device: string; Timing: string; Speedup: float }
let records = 
    [|
        { Device = "CPU"; Timing = sprintf "%A" cpuTime; Speedup =  1.0}
        { Device = "GPU"; Timing = sprintf "%A" gpuTime; Speedup =  cpuTime.TotalMilliseconds / gpuTime.TotalMilliseconds}
    |]

records |> Util.Table

sprintf "%A <-> %A  [x%.2f]" cpuTime gpuTime (cpuTime.TotalMilliseconds / gpuTime.TotalMilliseconds)

Let's collect Gpu performance over different count sizes:

In [ ]:
let performanceGpuData =
    seq {
        let plot = 1000
        for countInMB in [100..100..800] do
            let count = countInMB * 1024 * 1024
            let _, _, time = ecdfGpu count plot genGpuNormal
            yield countInMB, time.TotalMilliseconds
    } |> Seq.toList

First we plot the time:

In [ ]:
Bar(x = (performanceGpuData |> List.map fst |> List.map (sprintf "%dM")),
    y = (performanceGpuData |> List.map snd))
|> Chart.Plot

Now we plot in term of million numbers per second:

In [ ]:
Bar(x = (performanceGpuData |> List.map fst |> List.map (sprintf "%dM")),
    y = (performanceGpuData |> List.map (fun (countInMB, ms) -> (float countInMB) / ms * 1000.0)))
|> Chart.Plot

Now we test CPU against GPU:

In [ ]:
let performanceCpuGpu count plot =
    let _, _, cpuTime = ecdfCpu count plot genCpuNormal
    let _, _, gpuTime = ecdfGpu count plot genGpuNormal
    let cpuTime = cpuTime.TotalMilliseconds
    let gpuTime = gpuTime.TotalMilliseconds
    let speedup = cpuTime / gpuTime
    cpuTime, gpuTime, speedup
    
let performanceCpuGpuData =
    seq {
        let plot = 1000
        for countInMB in [300; 500] do
            let count = countInMB * 1024 * 1024
            let cpuTime, gpuTime, speedup = performanceCpuGpu count plot
            yield countInMB, cpuTime, gpuTime, speedup
    } |> Seq.toList

We compare performance by different count size:

In [ ]:
seq {
    let x = performanceCpuGpuData |> List.map (fun (count, _, _, _) -> sprintf "%dM" count)
    let yCpu = performanceCpuGpuData |> List.map (fun (count, t, _, _) -> (float count) / t * 1000.0)
    let yGpu = performanceCpuGpuData |> List.map (fun (count, _, t, _) -> (float count) / t * 1000.0)
    yield Bar(x = x, y = yCpu, name = "CPU")
    yield Bar(x = x, y = yGpu, name = "GPU")
} |> Seq.toList |> Chart.Plot